In [1]:
from cnns_classes import t1_quantizedCNN, t2_quantizedCNN

target_fps_list = [500,1000,2000]

topologies = [
    {
        'id':1, 
        'tp_class':t1_quantizedCNN,
        'quant': [2,4,8]
    },
    {
        'id':2, 
        'tp_class':t2_quantizedCNN,
        'quant': [2,4,8]
    }]

In [24]:
headers = ['Hardware config','Total LUTs','Total FFs','BRAM']
PYNQ_Z1_max = ['', 53200, 106400, 140]

#headers = ['Hardware config','Total LUTs','Logic LUTs','LUTRAM','SRLs','FFs','RAMB36','RAMB18','DSP Blocks']
#PYNQ_Z1_max = ['', 53200, 53200, 53200, 53200, 106400, 140, 280, 220]

print(headers)

['Hardware config', 'Total LUTs', 'Total FFs', 'BRAM']


In [56]:
import numpy as np
import pandas as pd
import json
import os  
 
# results_finn = [headers]
results_vivado = [headers]
pre_data = [headers]
# diff_table = [headers]

for tp in topologies:
    for quant in tp['quant']:
        for target_fps in target_fps_list:
            hardware_config = f"t{tp['id']}w{quant}_{target_fps}fps"
            # diff_data = []
            filename = f"/home/artti/Desktop/finn/notebooks/sat6_cnn/builds/build_{hardware_config}/stitched_ip/finn_design_partition_util.rpt"
            with open(filename) as my_file:
                content = my_file.readlines() 
                data = [hardware_config]
                for line in content:                
                    if ("finn_design_wrapper" in line and "(top)" in line):
                        line_content = line.split()            
                        for word in line_content:
                            if (word.isnumeric()):
                                #use_percentage = int(word)*100/PYNQ_Z1_max[len(data)]
                                #use_percentage = float("{:.2f}".format(use_percentage))
                                data.append(word)
                                # diff_data.append(int(word))
                        break
                results_vivado.append(data)

            
                
            # filename = f"/home/artti/Desktop/finn/notebooks/sat6_cnn/builds/build_{hardware_config}/report/ooc_synth_and_timing.json"
            # with open(filename) as my_file:
            #     finn_data = json.load(my_file)
            #     area_data = [
            #         int(finn_data['LUT']),
            #         int(finn_data['LUT']- finn_data['LUTRAM']),
            #         int(finn_data['LUTRAM']),
            #         0,
            #         int(finn_data['FF']),
            #         int(finn_data['BRAM_36K']),
            #         int(finn_data['BRAM_18K']),
            #         int(finn_data['DSP'])
            #     ]
                
            #     results_finn.append([
            #         hardware_config,
            #         int(finn_data['LUT']),
            #         int(finn_data['LUT']- finn_data['LUTRAM']),
            #         int(finn_data['LUTRAM']),
            #         '0',
            #         int(finn_data['FF']),
            #         int(finn_data['BRAM_36K']),
            #         int(finn_data['BRAM_18K']),
            #         int(finn_data['DSP'])
            #     ])
                
            #     diff_data = np.subtract(diff_data,area_data)
            #     diff_data = list(diff_data)
                
            #     diff_data.insert(0, hardware_config)
            #     diff_table.append(diff_data)

for data in results_vivado[1:]:
    data.pop(4)
    data.pop(3)
    data.pop(2)    
    data[3] = str(int(data[3]) + int(data[4])/2)
    data.pop(4)
    data.pop(4)
    
index = 1;
for data in results_vivado[1:]:
    data[1] = f"{data[1]} ({int(data[1])*100/PYNQ_Z1_max[1]:.2f}%)"
    data[2] = f"{data[2]} ({int(data[2])*100/PYNQ_Z1_max[2]:.2f}%)"
    data[3] = f"{data[3]} ({float(data[3])*100/PYNQ_Z1_max[3]:.2f}%)"
    results_vivado[index] = data
    index += 1

# df_finn = pd.DataFrame(results_finn)
# print("(FINN) Hardware area results")
# print(df_finn.to_string(header=None, index=False))

# print()

df_vivado = pd.DataFrame(results_vivado)
print("(Vivado) Hardware area results")
print(df_vivado.to_string(header=None, index=False))

df_vivado.to_csv('./results/vivado_area_results_modified.csv') 

# print()

# df_diff = pd.DataFrame(diff_table)
# print("Hardware area results difference")
# print(df_diff.to_string(header=None, index=False))

(Vivado) Hardware area results
Hardware config     Total LUTs      Total FFs         BRAM
    t1w2_500fps   4697 (8.83%)   5939 (5.58%)  4.0 (2.86%)
   t1w2_1000fps   4780 (8.98%)   6121 (5.75%)  9.5 (6.79%)
   t1w2_2000fps   4816 (9.05%)   6239 (5.86%)  9.5 (6.79%)
    t1w4_500fps  6260 (11.77%)  10205 (9.59%)  5.0 (3.57%)
   t1w4_1000fps  6353 (11.94%)  10401 (9.78%) 10.5 (7.50%)
   t1w4_2000fps  6414 (12.06%)  10516 (9.88%) 10.5 (7.50%)
    t1w8_500fps 14144 (26.59%) 19532 (18.36%)  7.5 (5.36%)
   t1w8_1000fps 14225 (26.74%) 19741 (18.55%) 13.0 (9.29%)
   t1w8_2000fps 14390 (27.05%) 19897 (18.70%) 13.0 (9.29%)
    t2w2_500fps  6891 (12.95%)   4690 (4.41%)  6.0 (4.29%)
   t2w2_1000fps  6908 (12.98%)   4846 (4.55%) 11.5 (8.21%)
   t2w2_2000fps  6693 (12.58%)   4921 (4.62%) 10.5 (7.50%)
    t2w4_500fps  7335 (13.79%)   5751 (5.41%)  6.5 (4.64%)
   t2w4_1000fps  7357 (13.83%)   5920 (5.56%) 12.0 (8.57%)
   t2w4_2000fps  7383 (13.88%)   6004 (5.64%) 11.0 (7.86%)
    t2w8_500fps 19279 (36